In [ ]:
import json
import re

# JSON data containing LaTeX symbols and their English pronunciation
latex_to_english = {
    "구별 부호": {
        "\\dot{a}": "a dot",
        "\\ddot{a}": "a double dot",
        "\\acute{a}": "a acute",
        "\\grave{a}": "a grave",
        "\\check{a}": "a check",
        "\\breve{a}": "a breve",
        "\\tilde{a}": "a tilde",
        "\\bar{a}": "a bar",
        "\\hat{a}": "a hat",
        "\\widehat{a}": "a wide hat",
        "\\vec{a}": "a vector"
    },
    "산술함수": {
        "\\exp_a b": "a raised to the power of b",
        "\\exp b": "e raised to the power of b",
        "10^m": "10 raised to the power of m",
        "\\ln c": "natural logarithm of c",
        "\\lg d": "common logarithm of d",
        "\\log e": "logarithm of e",
        "\\log_{10} f": "base 10 logarithm of f"
    },
    "삼각함수": {
        "\\sin a": "sine of a",
        "\\cos b": "cosine of b",
        "\\tan c": "tangent of c",
        "\\cot d": "cotangent of d",
        "\\sec e": "secant of e",
        "\\csc f": "cosecant of f"
    },
    "역삼각함수": {
        "\\arcsin h": "arc sine of h",
        "\\arccos i": "arc cosine of i",
        "\\arctan j": "arc tangent of j"
    },
    "쌍곡함수": {
        "\\sinh k": "hyperbolic sine of k",
        "\\cosh l": "hyperbolic cosine of l",
        "\\tanh m": "hyperbolic tangent of m",
        "\\coth n": "hyperbolic cotangent of n"
    },
    "특수함수": {
        "\\operatorname{sh}\\,k": "hyperbolic sine of k",
        "\\operatorname{ch}\\,l": "hyperbolic cosine of l",
        "\\operatorname{th}\\,m": "hyperbolic tangent of m",
        "\\operatorname{argsh}\\,o": "area hyperbolic sine of o",
        "\\operatorname{argch}\\,p": "area hyperbolic cosine of p",
        "\\operatorname{argth}\\,q": "area hyperbolic tangent of q",
        "\\sgn r": "sign of r",
        "\\left\\vert s \\right\\vert": "absolute value of s"
    },
    "상한과 하한": {
        "\\min x": "minimum of x",
        "\\max y": "maximum of y",
        "\\inf s": "infimum of s",
        "\\sup t": "supremum of t"
    },
    "극한": {
        "\\lim u": "limit of u",
        "\\liminf v": "limit inferior of v",
        "\\limsup w": "limit superior of w"
    },
    "기타 수학 기호": {
        "\\dim p": "dimension of p",
        "\\deg q": "degree of q",
        "\\det m": "determinant of m",
        "\\ker\\phi": "kernel of phi"
    },
    "투영": {
        "\\Pr j": "probability of j",
        "\\hom l": "homomorphism of l",
        "\\lVert z \\rVert": "norm of z",
        "\\arg z": "argument of z"
    },
    "미분": {
        "dt": "d t",
        "\\operatorname{d}\\!t": "differential d t",
        "\\partial t": "partial t",
        "\\nabla\\psi": "nabla psi"
    },
    "미분 연산": {
        "dy/dx": "dy over dx",
        "\\operatorname{d}\\!y/\\operatorname{d}\\!x": "differential of y over differential of x",
        "{dy \\over dx}": "dy over dx",
        "{\\operatorname{d}\\!y\\over\\operatorname{d}\\!x}": "differential of y over differential of x",
        "{\\partial^2\\over\\partial x_1\\partial x_2}y": "second partial derivative of y with respect to x1 and x2"
    },
    "기타": {
        "\\prime": "prime",
        "\\backprime": "back prime",
        "f'": "f prime",
        "f''": "f double prime",
        "f^{(3)}": "f triple prime",
        "\\dot y": "y dot",
        "\\ddot y": "y double dot"
    },
    "유사 문자 기호": {
        "\\infty": "infinity",
        "\\aleph": "aleph",
        "\\complement": "complement",
        "\\backepsilon": "back epsilon",
        "\\eth": "eth",
        "\\Finv": "F inverse",
        "\\hbar": "h bar"
    },
    "기타 문자 기호": {
        "\\Im": "imaginary part",
        "\\imath": "i math",
        "\\jmath": "j math",
        "\\Bbbk": "blackboard bold k",
        "\\ell": "ell",
        "\\mho": "mho",
        "\\wp": "Weierstrass p",
        "\\Re": "real part",
        "\\circledS": "circled S"
    },
    "모듈러 연산": {
        "s_k \\equiv 0 \\pmod{m}": "s sub k is congruent to 0 modulo m",
        "a\\,\\bmod\\,b": "a mod b",
        "\\gcd(m, n)": "greatest common divisor of m and n",
        "\\operatorname{lcm}(m, n)": "least common multiple of m and n"
    },
    "근호": {
        "\\surd": "surd",
        "\\sqrt{2}": "square root of 2",
        "\\sqrt[n]{}": "n-th root",
        "\\sqrt[3]{x^3+y^3 \\over 2}": "cube root of (x cubed plus y cubed over 2)"
    },
    "조합론": {
        "_{n}\\mathrm{P}_{k}": "n P k",
        "_{n}\\mathrm{C}_{k}": "n C k",
        "_{n}\\mathrm{\\Pi}_{k}": "n Pi k",
        "_{n}\\mathrm{H}_{k}": "n H k",
        "P(n,k)": "P of n and k",
        "S(n,k)": "S of n and k",
        "\\mathrm{P}_{k}^{n}": "P sub k superscript n"
    },
    "연산자": {
        "+": "plus",
        "-": "minus",
        "\\pm": "plus or minus",
        "\\mp": "minus or plus",
        "\\dotplus": "dot plus",
        "\\times": "times",
        "\\div": "divided by",
        "\\divideontimes": "divide on times",
        "/": "slash",
        "\\backslash": "backslash",
        "\\cdot": "dot",
        "*": "asterisk",
        "\\ast": "ast",
        "\\star": "star",
        "\\circ": "circ",
        "\\bullet": "bullet",
        "\\boxplus": "box plus",
        "\\boxminus": "box minus",
        "\\boxtimes": "box times",
        "\\boxdot": "box dot",
        "\\oplus": "o plus",
        "\\ominus": "o minus",
        "\\otimes": "o times",
        "\\oslash": "o slash",
        "\\odot": "o dot",
        "\\circleddash": "circled dash",
        "\\circledcirc": "circled circ",
        "\\circledast": "circled ast",
        "\\bigoplus": "big o plus",
        "\\bigotimes": "big o times",
        "\\bigodot": "big o dot"
    },
    "집합": {
        "\\{ \\}": "curly braces",
        "\\emptyset": "empty set",
        "\\varnothing": "var nothing",
        "\\in": "element of",
        "\\notin": "not element of",
        "\\ni": "contains as member",
        "\\not\\ni": "does not contain as member",
        "\\cap": "intersection",
        "\\Cup": "cup",
        "\\bigcap": "big cap",
        "\\bigcup": "big cup",
        "\\setminus": "set minus",
        "\\times": "times",
        "\\subset": "subset",
        "\\not\\subset": "not subset",
        "\\Subset": "subset of",
        "\\sqsubset": "square subset",
        "\\supset": "superset",
        "\\not\\supset": "not superset",
        "\\Supset": "superset of",
        "\\sqsupset": "square superset",
        "\\subseteq": "subset equal",
        "\\nsubseteq": "not subset equal",
        "\\subsetneq": "subset not equal",
        "\\varsubsetneq": "var subset not equal",
        "\\supseteq": "superset equal",
        "\\nsupseteq": "not superset equal",
        "\\supsetneq": "superset not equal",
        "\\varsupsetneq": "var superset not equal"
    },
    "관계": {
        "=": "equals",
        "\\ne": "not equals",
        "\\neq": "not equals",
        "\\equiv": "equivalent",
        "\\not\\equiv": "not equivalent",
        "\\doteq": "dot equals",
        "\\doteqdot": "dot equals dot",
        "\\sim": "similar",
        "\\nsim": "not similar",
        "\\cong": "congruent",
        "\\ncong": "not congruent",
        "\\approx": "approximately",
        "\\propto": "proportional to",
        "<": "less than",
        "\\nless": "not less than",
        ">": "greater than",
        "\\ngtr": "not greater than",
        "\\leq": "less than or equal to",
        "\\geq": "greater than or equal to",
        "\\prec": "precedes",
        "\\nprec": "does not precede",
        "\\preceq": "precedes or equals",
        "\\npreceq": "does not precede or equal",
        "\\succ": "succeeds",
        "\\nsucc": "does not succeed",
        "\\succeq": "succeeds or equals",
        "\\nsucceq": "does not succeed or equal"
    },
    "기하": {
        "\\parallel": "parallel",
        "\\nparallel": "not parallel",
        "\\perp": "perpendicular",
        "\\angle": "angle",
        "\\sphericalangle": "spherical angle",
        "\\measuredangle": "measured angle",
        "\\Box": "box",
        "\\blacksquare": "black square",
        "\\diamond": "diamond",
        "\\bigstar": "big star",
        "\\triangle": "triangle",
        "\\vartriangle": "var triangle",
        "\\blacktriangle": "black triangle"
    },
    "기타 기호": {
        "\\forall": "for all",
        "\\exists": "there exists",
        "\\nexists": "there does not exist",
        "\\therefore": "therefore",
        "\\because": "because",
        "\\lor": "logical or",
        "\\land": "logical and",
        "\\lnot": "logical not",
        "\\vdash": "proves",
        "\\dashv": "is proved by",
        "\\models": "models",
        "\\ulcorner": "upper left corner",
        "\\urcorner": "upper right corner",
        "\\llcorner": "lower left corner",
        "\\lrcorner": "lower right corner",
        "\\Rightarrow": "implies",
        "\\Leftarrow": "implied by",
        "\\Leftrightarrow": "if and only if",
        "\\Uparrow": "upwards arrow",
        "\\Downarrow": "downwards arrow",
        "\\Updownarrow": "upwards and downwards arrow",
        "\\rightarrow": "right arrow",
        "\\leftarrow": "left arrow",
        "\\leftrightarrow": "left and right arrow",
        "\\uparrow": "up arrow",
        "\\downarrow": "down arrow",
        "\\updownarrow": "up and down arrow",
        "\\mapsto": "maps to",
        "\\longmapsto": "long maps to"
    },
    "특수 기호": {
        "\\amalg": "amalgamation",
        "\\P": "paragraph",
        "\\S": "section",
        "\\%": "percent",
        "\\dagger": "dagger",
        "\\ddagger": "double dagger",
        "\\ldots": "ellipsis",
        "\\cdots": "centered dots",
        "\\smile": "smile",
        "\\frown": "frown",
        "\\wr": "wreath product",
        "\\triangleleft": "triangle left",
        "\\triangleright": "triangle right",
        "\\diamondsuit": "diamond suit",
        "\\heartsuit": "heart suit",
        "\\clubsuit": "club suit",
        "\\spadesuit": "spade suit",
        "\\flat": "flat",
        "\\natural": "natural",
        "\\sharp": "sharp"
    },
    "삭제표시": {
        "\\cancel{1}": "cancel 1"
    },
    "색깔": {
        "{\\color{Blue}x^2}+{\\color{Red}2x}-{\\color{Green}1}": "x squared in blue plus 2x in red minus 1 in green"
    },
    "기타": {
        "\\diagup": "diagonal up",
        "\\diagdown": "diagonal down",
        "\\centerdot": "center dot",
        "\\ltimes": "left times",
        "\\rtimes": "right times",
        "\\leftthreetimes": "left three times",
        "\\rightthreetimes": "right three times",
        "\\eqcirc": "equals circ",
        "\\circeq": "circ equals",
        "\\triangleq": "triangle equals",
        "\\bumpeq": "bump equals",
        "\\Bumpeq": "big bump equals",
        "\\doteqdot": "dot equals dot",
        "\\risingdotseq": "rising dots equal",
        "\\fallingdotseq": "falling dots equal",
        "\\intercal": "intercal",
        "\\barwedge": "bar wedge",
        "\\veebar": "vee bar",
        "\\doublebarwedge": "double bar wedge",
        "\\between": "between",
        "\\pitchfork": "pitchfork",
        "\\vartriangleleft": "var triangle left",
        "\\ntriangleleft": "not triangle left",
        "\\vartriangleright": "var triangle right",
        "\\ntriangleright": "not triangle right",
        "\\trianglelefteq": "triangle left equals",
        "\\ntrianglelefteq": "not triangle left equals",
        "\\trianglerighteq": "triangle right equals",
        "\\ntrianglerighteq": "not triangle right equals"
    },
    "위, 아래, 전치, 후치 첨자": {
        "a^2": "a squared",
        "y_m": "y sub m",
        "x_s-x_D": "x sub s minus x sub D",
        "a^{2+2}": "a to the power of 2 plus 2",
        "a_{i,j}": "a sub i j",
        "x_2^3": "x sub 2 cubed",
        "\\overset{x}{P}": "P overset x",
        "{}_{b}^{a}X": "X sub b superscript a",
        "_{c}^{a}Z_{d}^{b}": "Z sub c superscript a sub d superscript b",
        "\\underset{y}{\\overset{x}{_{c}^{a}Z_{d}^{b}}}": "Z sub c superscript a sub d superscript b overset x underset y",
        "x'": "x prime",
        "x^{\\prime}": "x prime",
        "x\\prime": "x prime"
    },
    "시그마": {
        "\\sum_{k=1}^N k^2": "sum from k equals 1 to N of k squared"
    },
    "곱기호": {
        "\\prod_{i=1}^N x_i": "product from i equals 1 to N of x sub i"
    },
    "극한": {
        "\\lim_{n \\to \\infty}x_n": "limit as n approaches infinity of x sub n"
    },
    "적분": {
        "\\int_{-N}^{N} e^x\\, dx": "integral from negative N to N of e raised to the power of x dx"
    },
    "선적분": {
        "\\oint_{C} x^3\\, dx + 4y^2\\, dy": "contour integral over C of x cubed dx plus 4y squared dy"
    },
    "분수, 행렬, 여러행": {
        "\\frac{2}{4}": "2 over 4",
        "{n \\choose k}": "n choose k",
        "\\begin{pmatrix} x & y \\\\ z & v \\end{pmatrix}": "matrix with elements x y on the first row, z v on the second row",
        "\\begin{bmatrix} 0 & \\cdots & 0 \\\\ \\vdots & \\ddots & \\vdots \\\\ 0 & \\cdots & 0 \\end{bmatrix}": "b matrix with elements 0 to 0, vertical dots to ddots to vertical dots, 0 to 0",
        "\\begin{Bmatrix} x & y \\\\ z & v \\end{Bmatrix}": "curly braces matrix with elements x y on the first row, z v on the second row",
        "\\begin{vmatrix} x & y \\\\ z & v \\end{vmatrix}": "vertical bar matrix with elements x y on the first row, z v on the second row",
        "\\begin{Vmatrix} x & y \\\\ z & v \\end{Vmatrix}": "double vertical bar matrix with elements x y on the first row, z v on the second row"
    },
    "경우 나누기": {
        "f(n)=\\begin{cases} n/2, & \\mbox{if }n\\mbox{ is even} \\\\ 3n+1, & \\mbox{if }n\\mbox{ is odd} \\end{cases}": "f of n equals n over 2 if n is even, 3n plus 1 if n is odd"
    },
    "두줄 이상의 방정식": {
        "\\begin{matrix} f(n+1) &=& (n+1)^2 \\\\ &=& n^2 + 2n + 1 \\end{matrix}": "f of n plus 1 equals (n plus 1) squared equals n squared plus 2n plus 1"
    },
    "글꼴": {
        "그리스어": {
            "\\Alpha": "Alpha",
            "\\Beta": "Beta",
            "\\Gamma": "Gamma",
            "\\Delta": "Delta",
            "\\Epsilon": "Epsilon",
            "\\Zeta": "Zeta",
            "\\Eta": "Eta",
            "\\Theta": "Theta",
            "\\Iota": "Iota",
            "\\Kappa": "Kappa",
            "\\Lambda": "Lambda",
            "\\Mu": "Mu",
            "\\Nu": "Nu",
            "\\Xi": "Xi",
            "\\Pi": "Pi",
            "\\Rho": "Rho",
            "\\Sigma": "Sigma",
            "\\Tau": "Tau",
            "\\Upsilon": "Upsilon",
            "\\Phi": "Phi",
            "\\Chi": "Chi",
            "\\Psi": "Psi",
            "\\Omega": "Omega",
            "\\alpha": "alpha",
            "\\beta": "beta",
            "\\gamma": "gamma",
            "\\delta": "delta",
            "\\epsilon": "epsilon",
            "\\zeta": "zeta",
            "\\eta": "eta",
            "\\theta": "theta",
            "\\iota": "iota",
            "\\kappa": "kappa",
            "\\lambda": "lambda",
            "\\mu": "mu",
            "\\nu": "nu",
            "\\xi": "xi",
            "\\pi": "pi",
            "\\rho": "rho",
            "\\sigma": "sigma",
            "\\tau": "tau",
            "\\upsilon": "upsilon",
            "\\phi": "phi",
            "\\chi": "chi",
            "\\psi": "psi",
            "\\omega": "omega",
            "\\varepsilon": "varepsilon",
            "\\digamma": "digamma",
            "\\varkappa": "varkappa",
            "\\varpi": "varpi",
            "\\varrho": "varrho",
            "\\varsigma": "varsigma",
            "\\vartheta": "vartheta",
            "\\varphi": "varphi"
        },
        "히브리어": {
            "\\aleph": "aleph",
            "\\beth": "beth",
            "\\gimel": "gimel",
            "\\daleth": "daleth"
        },
        "칠판체 로마자": {
            "\\mathbb{A}": "blackboard bold A",
            "\\mathbb{B}": "blackboard bold B",
            "\\mathbb{C}": "blackboard bold C",
            "\\mathbb{D}": "blackboard bold D",
            "\\mathbb{E}": "blackboard bold E",
            "\\mathbb{F}": "blackboard bold F",
            "\\mathbb{G}": "blackboard bold G",
            "\\mathbb{H}": "blackboard bold H",
            "\\mathbb{I}": "blackboard bold I",
            "\\mathbb{J}": "blackboard bold J",
            "\\mathbb{K}": "blackboard bold K",
            "\\mathbb{L}": "blackboard bold L",
            "\\mathbb{M}": "blackboard bold M",
            "\\mathbb{N}": "blackboard bold N",
            "\\mathbb{O}": "blackboard bold O",
            "\\mathbb{P}": "blackboard bold P",
            "\\mathbb{Q}": "blackboard bold Q",
            "\\mathbb{R}": "blackboard bold R",
            "\\mathbb{S}": "blackboard bold S",
            "\\mathbb{T}": "blackboard bold T",
            "\\mathbb{U}": "blackboard bold U",
            "\\mathbb{V}": "blackboard bold V",
            "\\mathbb{W}": "blackboard bold W",
            "\\mathbb{X}": "blackboard bold X",
            "\\mathbb{Y}": "blackboard bold Y",
            "\\mathbb{Z}": "blackboard bold Z"
        },
        "볼드체 로마자": {
            "\\mathbf{A}": "bold A",
            "\\mathbf{B}": "bold B",
            "\\mathbf{C}": "bold C",
            "\\mathbf{D}": "bold D",
            "\\mathbf{E}": "bold E",
            "\\mathbf{F}": "bold F",
            "\\mathbf{G}": "bold G",
            "\\mathbf{H}": "bold H",
            "\\mathbf{I}": "bold I",
            "\\mathbf{J}": "bold J",
            "\\mathbf{K}": "bold K",
            "\\mathbf{L}": "bold L",
            "\\mathbf{M}": "bold M",
            "\\mathbf{N}": "bold N",
            "\\mathbf{O}": "bold O",
            "\\mathbf{P}": "bold P",
            "\\mathbf{Q}": "bold Q",
            "\\mathbf{R}": "bold R",
            "\\mathbf{S}": "bold S",
            "\\mathbf{T}": "bold T",
            "\\mathbf{U}": "bold U",
            "\\mathbf{V}": "bold V",
            "\\mathbf{W}": "bold W",
            "\\mathbf{X}": "bold X",
            "\\mathbf{Y}": "bold Y",
            "\\mathbf{Z}": "bold Z",
            "\\mathbf{a}": "bold a",
            "\\mathbf{b}": "bold b",
            "\\mathbf{c}": "bold c",
            "\\mathbf{d}": "bold d",
            "\\mathbf{e}": "bold e",
            "\\mathbf{f}": "bold f",
            "\\mathbf{g}": "bold g",
            "\\mathbf{h}": "bold h",
            "\\mathbf{i}": "bold i",
            "\\mathbf{j}": "bold j",
            "\\mathbf{k}": "bold k",
            "\\mathbf{l}": "bold l",
            "\\mathbf{m}": "bold m",
            "\\mathbf{n}": "bold n",
            "\\mathbf{o}": "bold o",
            "\\mathbf{p}": "bold p",
            "\\mathbf{q}": "bold q",
            "\\mathbf{r}": "bold r",
            "\\mathbf{s}": "bold s",
            "\\mathbf{t}": "bold t",
            "\\mathbf{u}": "bold u",
            "\\mathbf{v}": "bold v",
            "\\mathbf{w}": "bold w",
            "\\mathbf{x}": "bold x",
            "\\mathbf{y}": "bold y",
            "\\mathbf{z}": "bold z",
            "\\mathbf{0}": "bold 0",
            "\\mathbf{1}": "bold 1",
            "\\mathbf{2}": "bold 2",
            "\\mathbf{3}": "bold 3",
            "\\mathbf{4}": "bold 4",
            "\\mathbf{5}": "bold 5",
            "\\mathbf{6}": "bold 6",
            "\\mathbf{7}": "bold 7",
            "\\mathbf{8}": "bold 8",
            "\\mathbf{9}": "bold 9"
        },
        "볼드체 그리스어": {
            "\\boldsymbol{\\Alpha}": "bold Alpha",
            "\\boldsymbol{\\Beta}": "bold Beta",
            "\\boldsymbol{\\Gamma}": "bold Gamma",
            "\\boldsymbol{\\Delta}": "bold Delta",
            "\\boldsymbol{\\Epsilon}": "bold Epsilon",
            "\\boldsymbol{\\Zeta}": "bold Zeta",
            "\\boldsymbol{\\Eta}": "bold Eta",
            "\\boldsymbol{\\Theta}": "bold Theta",
            "\\boldsymbol{\\Iota}": "bold Iota",
            "\\boldsymbol{\\Kappa}": "bold Kappa",
            "\\boldsymbol{\\Lambda}": "bold Lambda",
            "\\boldsymbol{\\Mu}": "bold Mu",
            "\\boldsymbol{\\Nu}": "bold Nu",
            "\\boldsymbol{\\Xi}": "bold Xi",
            "\\boldsymbol{\\Pi}": "bold Pi",
            "\\boldsymbol{\\Rho}": "bold Rho",
            "\\boldsymbol{\\Sigma}": "bold Sigma",
            "\\boldsymbol{\\Tau}": "bold Tau",
            "\\boldsymbol{\\Upsilon}": "bold Upsilon",
            "\\boldsymbol{\\Phi}": "bold Phi",
            "\\boldsymbol{\\Chi}": "bold Chi",
            "\\boldsymbol{\\Psi}": "bold Psi",
            "\\boldsymbol{\\Omega}": "bold Omega",
            "\\boldsymbol{\\alpha}": "bold alpha",
            "\\boldsymbol{\\beta}": "bold beta",
            "\\boldsymbol{\\gamma}": "bold gamma",
            "\\boldsymbol{\\delta}": "bold delta",
            "\\boldsymbol{\\epsilon}": "bold epsilon",
            "\\boldsymbol{\\zeta}": "bold zeta",
            "\\boldsymbol{\\eta}": "bold eta",
            "\\boldsymbol{\\theta}": "bold theta",
            "\\boldsymbol{\\iota}": "bold iota",
            "\\boldsymbol{\\kappa}": "bold kappa",
            "\\boldsymbol{\\lambda}": "bold lambda",
            "\\boldsymbol{\\mu}": "bold mu",
            "\\boldsymbol{\\nu}": "bold nu",
            "\\boldsymbol{\\xi}": "bold xi",
            "\\boldsymbol{\\pi}": "bold pi",
            "\\boldsymbol{\\rho}": "bold rho",
            "\\boldsymbol{\\sigma}": "bold sigma",
            "\\boldsymbol{\\tau}": "bold tau",
            "\\boldsymbol{\\upsilon}": "bold upsilon",
            "\\boldsymbol{\\phi}": "bold phi",
            "\\boldsymbol{\\chi}": "bold chi",
            "\\boldsymbol{\\psi}": "bold psi",
            "\\boldsymbol{\\omega}": "bold omega",
            "\\boldsymbol{\\varepsilon}": "bold varepsilon",
            "\\boldsymbol{\\digamma}": "bold digamma",
            "\\boldsymbol{\\varkappa}": "bold varkappa",
            "\\boldsymbol{\\varpi}": "bold varpi",
            "\\boldsymbol{\\varrho}": "bold varrho",
            "\\boldsymbol{\\varsigma}": "bold varsigma",
            "\\boldsymbol{\\vartheta}": "bold vartheta",
            "\\boldsymbol{\\varphi}": "bold varphi"
        },
        "기울임체 로마자": {
            "\\mathit{0}": "italic 0",
            "\\mathit{1}": "italic 1",
            "\\mathit{2}": "italic 2",
            "\\mathit{3}": "italic 3",
            "\\mathit{4}": "italic 4",
            "\\mathit{5}": "italic 5",
            "\\mathit{6}": "italic 6",
            "\\mathit{7}": "italic 7",
            "\\mathit{8}": "italic 8",
            "\\mathit{9}": "italic 9"
        },
        "기울임체 그리스어": {
            "\\mathit{\\Alpha}": "italic Alpha",
            "\\mathit{\\Beta}": "italic Beta",
            "\\mathit{\\Gamma}": "italic Gamma",
            "\\mathit{\\Delta}": "italic Delta",
            "\\mathit{\\Epsilon}": "italic Epsilon",
            "\\mathit{\\Zeta}": "italic Zeta",
            "\\mathit{\\Eta}": "italic Eta",
            "\\mathit{\\Theta}": "italic Theta",
            "\\mathit{\\Iota}": "italic Iota",
            "\\mathit{\\Kappa}": "italic Kappa",
            "\\mathit{\\Lambda}": "italic Lambda",
            "\\mathit{\\Mu}": "italic Mu",
            "\\mathit{\\Nu}": "italic Nu",
            "\\mathit{\\Xi}": "italic Xi",
            "\\mathit{\\Pi}": "italic Pi",
            "\\mathit{\\Rho}": "italic Rho",
            "\\mathit{\\Sigma}": "italic Sigma",
            "\\mathit{\\Tau}": "italic Tau",
            "\\mathit{\\Upsilon}": "italic Upsilon",
            "\\mathit{\\Phi}": "italic Phi",
            "\\mathit{\\Chi}": "italic Chi",
            "\\mathit{\\Psi}": "italic Psi",
            "\\mathit{\\Omega}": "italic Omega"
        },
        "로만체": {
            "\\mathrm{A}": "roman A",
            "\\mathrm{B}": "roman B",
            "\\mathrm{C}": "roman C",
            "\\mathrm{D}": "roman D",
            "\\mathrm{E}": "roman E",
            "\\mathrm{F}": "roman F",
            "\\mathrm{G}": "roman G",
            "\\mathrm{H}": "roman H",
            "\\mathrm{I}": "roman I",
            "\\mathrm{J}": "roman J",
            "\\mathrm{K}": "roman K",
            "\\mathrm{L}": "roman L",
            "\\mathrm{M}": "roman M",
            "\\mathrm{N}": "roman N",
            "\\mathrm{O}": "roman O",
            "\\mathrm{P}": "roman P",
            "\\mathrm{Q}": "roman Q",
            "\\mathrm{R}": "roman R",
            "\\mathrm{S}": "roman S",
            "\\mathrm{T}": "roman T",
            "\\mathrm{U}": "roman U",
            "\\mathrm{V}": "roman V",
            "\\mathrm{W}": "roman W",
            "\\mathrm{X}": "roman X",
            "\\mathrm{Y}": "roman Y",
            "\\mathrm{Z}": "roman Z",
            "\\mathrm{a}": "roman a",
            "\\mathrm{b}": "roman b",
            "\\mathrm{c}": "roman c",
            "\\mathrm{d}": "roman d",
            "\\mathrm{e}": "roman e",
            "\\mathrm{f}": "roman f",
            "\\mathrm{g}": "roman g",
            "\\mathrm{h}": "roman h",
            "\\mathrm{i}": "roman i",
            "\\mathrm{j}": "roman j",
            "\\mathrm{k}": "roman k",
            "\\mathrm{l}": "roman l",
            "\\mathrm{m}": "roman m",
            "\\mathrm{n}": "roman n",
            "\\mathrm{o}": "roman o",
            "\\mathrm{p}": "roman p",
            "\\mathrm{q}": "roman q",
            "\\mathrm{r}": "roman r",
            "\\mathrm{s}": "roman s",
            "\\mathrm{t}": "roman t",
            "\\mathrm{u}": "roman u",
            "\\mathrm{v}": "roman v",
            "\\mathrm{w}": "roman w",
            "\\mathrm{x}": "roman x",
            "\\mathrm{y}": "roman y",
            "\\mathrm{z}": "roman z",
            "\\mathrm{0}": "roman 0",
            "\\mathrm{1}": "roman 1",
            "\\mathrm{2}": "roman 2",
            "\\mathrm{3}": "roman 3",
            "\\mathrm{4}": "roman 4",
            "\\mathrm{5}": "roman 5",
            "\\mathrm{6}": "roman 6",
            "\\mathrm{7}": "roman 7",
            "\\mathrm{8}": "roman 8",
            "\\mathrm{9}": "roman 9"
        },
        "산세리프체": {
            "\\mathsf{A}": "sans serif A",
            "\\mathsf{B}": "sans serif B",
            "\\mathsf{C}": "sans serif C",
            "\\mathsf{D}": "sans serif D",
            "\\mathsf{E}": "sans serif E",
            "\\mathsf{F}": "sans serif F",
            "\\mathsf{G}": "sans serif G",
            "\\mathsf{H}": "sans serif H",
            "\\mathsf{I}": "sans serif I",
            "\\mathsf{J}": "sans serif J",
            "\\mathsf{K}": "sans serif K",
            "\\mathsf{L}": "sans serif L",
            "\\mathsf{M}": "sans serif M",
            "\\mathsf{N}": "sans serif N",
            "\\mathsf{O}": "sans serif O",
            "\\mathsf{P}": "sans serif P",
            "\\mathsf{Q}": "sans serif Q",
            "\\mathsf{R}": "sans serif R",
            "\\mathsf{S}": "sans serif S",
            "\\mathsf{T}": "sans serif T",
            "\\mathsf{U}": "sans serif U",
            "\\mathsf{V}": "sans serif V",
            "\\mathsf{W}": "sans serif W",
            "\\mathsf{X}": "sans serif X",
            "\\mathsf{Y}": "sans serif Y",
            "\\mathsf{Z}": "sans serif Z",
            "\\mathsf{a}": "sans serif a",
            "\\mathsf{b}": "sans serif b",
            "\\mathsf{c}": "sans serif c",
            "\\mathsf{d}": "sans serif d",
            "\\mathsf{e}": "sans serif e",
            "\\mathsf{f}": "sans serif f",
            "\\mathsf{g}": "sans serif g",
            "\\mathsf{h}": "sans serif h",
            "\\mathsf{i}": "sans serif i",
            "\\mathsf{j}": "sans serif j",
            "\\mathsf{k}": "sans serif k",
            "\\mathsf{l}": "sans serif l",
            "\\mathsf{m}": "sans serif m",
            "\\mathsf{n}": "sans serif n",
            "\\mathsf{o}": "sans serif o",
            "\\mathsf{p}": "sans serif p",
            "\\mathsf{q}": "sans serif q",
            "\\mathsf{r}": "sans serif r",
            "\\mathsf{s}": "sans serif s",
            "\\mathsf{t}": "sans serif t",
            "\\mathsf{u}": "sans serif u",
            "\\mathsf{v}": "sans serif v",
            "\\mathsf{w}": "sans serif w",
            "\\mathsf{x}": "sans serif x",
            "\\mathsf{y}": "sans serif y",
            "\\mathsf{z}": "sans serif z",
            "\\mathsf{0}": "sans serif 0",
            "\\mathsf{1}": "sans serif 1",
            "\\mathsf{2}": "sans serif 2",
            "\\mathsf{3}": "sans serif 3",
            "\\mathsf{4}": "sans serif 4",
            "\\mathsf{5}": "sans serif 5",
            "\\mathsf{6}": "sans serif 6",
            "\\mathsf{7}": "sans serif 7",
            "\\mathsf{8}": "sans serif 8",
            "\\mathsf{9}": "sans serif 9"
        },
        "산세리프체 그리스어": {
            "\\mathsf{\\Alpha}": "sans serif Alpha",
            "\\mathsf{\\Beta}": "sans serif Beta",
            "\\mathsf{\\Gamma}": "sans serif Gamma",
            "\\mathsf{\\Delta}": "sans serif Delta",
            "\\mathsf{\\Epsilon}": "sans serif Epsilon",
            "\\mathsf{\\Zeta}": "sans serif Zeta",
            "\\mathsf{\\Eta}": "sans serif Eta",
            "\\mathsf{\\Theta}": "sans serif Theta",
            "\\mathsf{\\Iota}": "sans serif Iota",
            "\\mathsf{\\Kappa}": "sans serif Kappa",
            "\\mathsf{\\Lambda}": "sans serif Lambda",
            "\\mathsf{\\Mu}": "sans serif Mu",
            "\\mathsf{\\Nu}": "sans serif Nu",
            "\\mathsf{\\Xi}": "sans serif Xi",
            "\\mathsf{\\Pi}": "sans serif Pi",
            "\\mathsf{\\Rho}": "sans serif Rho",
            "\\mathsf{\\Sigma}": "sans serif Sigma",
            "\\mathsf{\\Tau}": "sans serif Tau",
            "\\mathsf{\\Upsilon}": "sans serif Upsilon",
            "\\mathsf{\\Phi}": "sans serif Phi",
            "\\mathsf{\\Chi}": "sans serif Chi",
            "\\mathsf{\\Psi}": "sans serif Psi",
            "\\mathsf{\\Omega}": "sans serif Omega"
        },
        "흘림체": {
            "\\mathcal{A}": "calligraphic A",
            "\\mathcal{B}": "calligraphic B",
            "\\mathcal{C}": "calligraphic C",
            "\\mathcal{D}": "calligraphic D",
            "\\mathcal{E}": "calligraphic E",
            "\\mathcal{F}": "calligraphic F",
            "\\mathcal{G}": "calligraphic G",
            "\\mathcal{H}": "calligraphic H",
            "\\mathcal{I}": "calligraphic I",
            "\\mathcal{J}": "calligraphic J",
            "\\mathcal{K}": "calligraphic K",
            "\\mathcal{L}": "calligraphic L",
            "\\mathcal{M}": "calligraphic M",
            "\\mathcal{N}": "calligraphic N",
            "\\mathcal{O}": "calligraphic O",
            "\\mathcal{P}": "calligraphic P",
            "\\mathcal{Q}": "calligraphic Q",
            "\\mathcal{R}": "calligraphic R",
            "\\mathcal{S}": "calligraphic S",
            "\\mathcal{T}": "calligraphic T",
            "\\mathcal{U}": "calligraphic U",
            "\\mathcal{V}": "calligraphic V",
            "\\mathcal{W}": "calligraphic W",
            "\\mathcal{X}": "calligraphic X",
            "\\mathcal{Y}": "calligraphic Y",
            "\\mathcal{Z}": "calligraphic Z"
        },
        "흑자체": {
            "\\mathfrak{A}": "fraktur A",
            "\\mathfrak{B}": "fraktur B",
            "\\mathfrak{C}": "fraktur C",
            "\\mathfrak{D}": "fraktur D",
            "\\mathfrak{E}": "fraktur E",
            "\\mathfrak{F}": "fraktur F",
            "\\mathfrak{G}": "fraktur G",
            "\\mathfrak{H}": "fraktur H",
            "\\mathfrak{I}": "fraktur I",
            "\\mathfrak{J}": "fraktur J",
            "\\mathfrak{K}": "fraktur K",
            "\\mathfrak{L}": "fraktur L",
            "\\mathfrak{M}": "fraktur M",
            "\\mathfrak{N}": "fraktur N",
            "\\mathfrak{O}": "fraktur O",
            "\\mathfrak{P}": "fraktur P",
            "\\mathfrak{Q}": "fraktur Q",
            "\\mathfrak{R}": "fraktur R",
            "\\mathfrak{S}": "fraktur S",
            "\\mathfrak{T}": "fraktur T",
            "\\mathfrak{U}": "fraktur U",
            "\\mathfrak{V}": "fraktur V",
            "\\mathfrak{W}": "fraktur W",
            "\\mathfrak{X}": "fraktur X",
            "\\mathfrak{Y}": "fraktur Y",
            "\\mathfrak{Z}": "fraktur Z",
            "\\mathfrak{a}": "fraktur a",
            "\\mathfrak{b}": "fraktur b",
            "\\mathfrak{c}": "fraktur c",
            "\\mathfrak{d}": "fraktur d",
            "\\mathfrak{e}": "fraktur e",
            "\\mathfrak{f}": "fraktur f",
            "\\mathfrak{g}": "fraktur g",
            "\\mathfrak{h}": "fraktur h",
            "\\mathfrak{i}": "fraktur i",
            "\\mathfrak{j}": "fraktur j",
            "\\mathfrak{k}": "fraktur k",
            "\\mathfrak{l}": "fraktur l",
            "\\mathfrak{m}": "fraktur m",
            "\\mathfrak{n}": "fraktur n",
            "\\mathfrak{o}": "fraktur o",
            "\\mathfrak{p}": "fraktur p",
            "\\mathfrak{q}": "fraktur q",
            "\\mathfrak{r}": "fraktur r",
            "\\mathfrak{s}": "fraktur s",
            "\\mathfrak{t}": "fraktur t",
            "\\mathfrak{u}": "fraktur u",
            "\\mathfrak{v}": "fraktur v",
            "\\mathfrak{w}": "fraktur w",
            "\\mathfrak{x}": "fraktur x",
            "\\mathfrak{y}": "fraktur y",
            "\\mathfrak{z}": "fraktur z",
            "\\mathfrak{0}": "fraktur 0",
            "\\mathfrak{1}": "fraktur 1",
            "\\mathfrak{2}": "fraktur 2",
            "\\mathfrak{3}": "fraktur 3",
            "\\mathfrak{4}": "fraktur 4",
            "\\mathfrak{5}": "fraktur 5",
            "\\mathfrak{6}": "fraktur 6",
            "\\mathfrak{7}": "fraktur 7",
            "\\mathfrak{8}": "fraktur 8",
            "\\mathfrak{9}": "fraktur 9"
        },
        "작은 글자": {
            "{\\scriptstyle\\text{abcdefghijklm}}": "scriptstyle a to m"
        }
    },
    "괄호 쓰기": {
        "( \\frac{1}{2} )": "parentheses one half",
        "\\left( \\frac{1}{2} \\right)": "left parentheses one half right parentheses",
        "\\left( A \\right)": "left parentheses A right parentheses",
        "\\left[ A \\right]": "left bracket A right bracket",
        "\\left\\{ A \\right\\}": "left curly brace A right curly brace",
        "\\left\\langle A \\right\\rangle": "left angle bracket A right angle bracket",
        "\\left| A \\right|": "left vertical bar A right vertical bar",
        "\\left. {A \\over B} \\right\\} \\to X": "left period A over B right curly brace to X",
        "\\lfloor\\sqrt{n}\\rfloor": "left floor square root of n right floor",
        "\\lceil\\sqrt{n}\\rceil": "left ceiling square root of n right ceiling"
    },
    "빈칸 조정": {
        "a \\qquad b": "a double quad space b",
        "a \\quad b": "a quad space b",
        "a\\ b": "a text space b",
        "a\\;b": "a large space b",
        "a\\,b": "a small space b",
        "ab": "a b",
        "a\\!b": "a negative space b"
    }
}

# Function to convert LaTeX to English pronunciation
def latex_to_english_pronunciation(latex_string, data):
    for section in data:
        for key, value in data[section].items():
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    latex_string = latex_string.replace(sub_key, sub_value)
            else:
                latex_string = latex_string.replace(key, value)
    return latex_string

# Example usage
latex_string = "\\frac{2}{4} + \\sin a + \\int_{-N}^{N} e^x\\, dx"
english_pronunciation = latex_to_english_pronunciation(latex_string, latex_to_english)
print(english_pronunciation)


TypeError: replace() argument 2 must be str, not dict